# Практика 8

### Отображение базы данных

In [1]:
from sqlalchemy import MetaData, Table, create_engine, select
from pprint import pprint
import warnings 
warnings.filterwarnings("ignore")

metadata=MetaData()
engine = create_engine("sqlite:///BD_Primer_SQLite.db")

metadata.reflect(bind=engine)

connection = engine.connect()

In [2]:
pprint(metadata.tables)

{'exam_marks': Table('exam_marks', MetaData(bind=None), Column('EXAM_ID', INTEGER(), table=<exam_marks>, primary_key=True, nullable=False), Column('STUDENT_ID', INTEGER(), table=<exam_marks>, nullable=False), Column('SUBJ_ID', INTEGER(), table=<exam_marks>, nullable=False), Column('MARK', SMALLINT(), table=<exam_marks>, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x7fdd86e10430>, for_update=False)), Column('EXAM_DATE', DATE(), table=<exam_marks>, nullable=False), schema=None),
 'lecturer': Table('lecturer', MetaData(bind=None), Column('LECTURER_ID', INTEGER(), table=<lecturer>, primary_key=True, nullable=False), Column('SURNAME', TEXT(), table=<lecturer>, nullable=False), Column('NAME', TEXT(), table=<lecturer>, nullable=False), Column('CITY', TEXT(), table=<lecturer>, nullable=False), Column('UNIV_ID', INTEGER(), table=<lecturer>, nullable=False), schema=None),
 'student': Table('student', MetaData(bind=None), Column('STUDENT_ID', INTEGER(), table=<stude

Убедимся, что отображение выполено. 

In [4]:
print(metadata.tables.keys())

dict_keys(['exam_marks', 'lecturer', 'student', 'subj_lect', 'subject', 'university'])


Создание объектов таблиц в цикле. 

In [6]:
tables=[]
for table in metadata.tables.keys():
    tables.append(metadata.tables[table])
exam_marks, lecturer, student, subj_lect, subject, university = tables

In [7]:
exam_marks

Table('exam_marks', MetaData(bind=None), Column('EXAM_ID', INTEGER(), table=<exam_marks>, primary_key=True, nullable=False), Column('STUDENT_ID', INTEGER(), table=<exam_marks>, nullable=False), Column('SUBJ_ID', INTEGER(), table=<exam_marks>, nullable=False), Column('MARK', SMALLINT(), table=<exam_marks>, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x7fdd86e10430>, for_update=False)), Column('EXAM_DATE', DATE(), table=<exam_marks>, nullable=False), schema=None)

Выбрать сведения о студентах, проживающих в городе, где расположен университет, в котором они учатся. 

In [11]:
sel = select([student])
sel = sel.where(student.c.CITY == (select([university.c.CITY])
                                  .where(university.c.UNIV_ID) == student.c.UNIV_ID))

In [14]:
rp = connection.execute(sel).fetchall()
print(rp)

[]


In [12]:
str(sel)

'SELECT student."STUDENT_ID", student."SURNAME", student."NAME", student."STIPEND", student."KURS", student."CITY", student."BIRTHDAY", student."UNIV_ID" \nFROM student \nWHERE student."CITY" = (student."UNIV_ID" = (SELECT university."CITY" \nFROM university \nWHERE university."UNIV_ID"))'